# Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

Описание датасета можно посмотреть
http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/

книга по nltk
http://www.nltk.org/book/ch02.html

Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [1]:
f = open('SMSSpamCollection.txt')
text = f.read()
text[:300]

"ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\nham\tOk lar... Joking wif u oni...\nspam\tFree entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 084528100"

используется в качестве разделителей \n \t 

проставляются метки классов 'ham', 'spam'

Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [30]:
import nltk

In [31]:
# Loading your own Corpus
#from nltk.corpus import PlaintextCorpusReader
#corpus_root = '/usr/share/dict'
#wordlists = PlaintextCorpusReader(corpus_root, '.*') [2]
#wordlists = PlaintextCorpusReader('/home/thomas/Jupyter', 'SMSSpamCollection.txt')

In [32]:
 #help(nltk.corpus.reader)

In [33]:
text = nltk.data.load('SMSSpamCollection.txt')

In [34]:
data = []
indexes = []
sentences_all = text.split('\n')

for i in range(len(sentences_all)):
    try:
        sentence = sentences_all[i]
        words = sentence.split("\t")
        data.append(words[1])
        indexes.append(words[0])
    except:
        pass

for j in range(len(indexes)):
    if indexes[j] == 'spam':
        indexes[j] = 1
    else:
        indexes[j] = 0
    
print sentences_all[5573]
print indexes[5573], data[5573]
print sentences_all[0]
print indexes[0], data[0] 

ham	Rofl. Its true to its name
0 Rofl. Its true to its name
ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
0 Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...


Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [35]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [36]:
c_vect = CountVectorizer()
X = c_vect.fit_transform(data)

Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. 

Установите random_state=2. 

Параметр cv задайте равным 10. 

В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [37]:
# это классификатор
from sklearn.linear_model import LogisticRegression
# это модуль для проверки качества
from sklearn.model_selection import cross_val_score
# пайплайн
from sklearn.pipeline import Pipeline

In [38]:
# функция возвращает пайплайн для текстовой классификации
pipeline = Pipeline([('c_vect', CountVectorizer()), ('log_reg', LogisticRegression())])

In [39]:
# проверяется качество классификаторов
print round(cross_val_score(pipeline, data, indexes, scoring='f1', cv=10).mean(), 1)

0.9


In [40]:
with open('c5_w3_1.txt', 'w') as f:
    f.write(str(0.9))

А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [41]:
# проверка качества классификатора на выше написанных текстах

texts = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
        "FreeMsg: Txt: claim your reward of 3 hours talk time",
        "Have you visited the last lecture on physics?",
        "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
        "Only 99$"]

pipeline.fit(data, indexes)

print pipeline.predict(texts)

[1 1 0 0 0]


In [45]:
with open('c5_w3_2.txt', 'w') as f:
    f.write(' '.join(map(str, [1, 1, 0, 0, 0])))

In [43]:
print pipeline.predict(["Fuck you",
                        "I love my mom",
                       "Bible is a collection of sacred texts",
                       "It is a spam msg",
                       "FreeMsg: Txt: By it for free now!"])

[0 0 0 0 1]


Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [44]:
ngrams = [(2,2), (3,3), (1,3)]

scores = []
for ngram in ngrams:
    pipeline = Pipeline([('count_vect', CountVectorizer(ngram_range=ngram)), 
                         ('log_reg', LogisticRegression())])
    score = cross_val_score(pipeline, data, indexes, scoring='f1', cv=10).mean()
    scores.append(round(score, 2))
    print '\nngram ', ngram
    print 'cross-val-score f1: ', score


ngram  (2, 2)
cross-val-score f1:  0.8224220664187133

ngram  (3, 3)
cross-val-score f1:  0.7250161555467377

ngram  (1, 3)
cross-val-score f1:  0.9251382558648837


In [46]:
with open('c5_w3_3.txt', 'w') as f:
    f.write(' '.join(map(str, scores)))

Повторите аналогичный выше описанному эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

In [50]:
from sklearn.naive_bayes import MultinomialNB

In [52]:
scores = []
for ngram in ngrams:
    pipeline = Pipeline([('count_vect', CountVectorizer(ngram_range=ngram)), 
                         ('mn_nb', MultinomialNB())])
    score = cross_val_score(pipeline, data, indexes, scoring='f1', cv=10).mean()
    scores.append(round(score, 2))
    print '\nngram ', ngram
    print 'cross-val-score f1: ', score


ngram  (2, 2)
cross-val-score f1:  0.9337279133513456

ngram  (3, 3)
cross-val-score f1:  0.871265305963816

ngram  (1, 3)
cross-val-score f1:  0.9472991994136064


In [54]:
ngrams = [(2,2), (3,3), (1,3)]
scores = []
for ngram in ngrams:
    X = CountVectorizer(ngram_range=ngram).fit_transform(data)
    mn_nb = MultinomialNB()
    score = cross_val_score(mn_nb, X, indexes, scoring='f1', cv=10).mean()
    scores.append(round(score, 2))
    print '\nngram ', ngram
    print 'cross-val-score f1: ', score


ngram  (2, 2)
cross-val-score f1:  0.6454554013558982

ngram  (3, 3)
cross-val-score f1:  0.37862343087618666

ngram  (1, 3)
cross-val-score f1:  0.8879054608894993


In [55]:
with open('c5_w3_4.txt', 'w') as f:
    f.write(' '.join(map(str, scores)))

Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

### Tf*Idf вместо частот слов:

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [59]:
%%time

pipeline = Pipeline([('tf_idf', TfidfVectorizer()),
                     ('log_reg', LogisticRegression())])
score = cross_val_score(pipeline, data, indexes, scoring='f1', cv=10).mean()
print 'Cross-val-score f1: ', score

Cross-val-score f1:  0.8785100455343396
CPU times: user 12.8 s, sys: 408 ms, total: 13.2 s
Wall time: 7.22 s


In [60]:
with open('c5_w3_5.txt', 'w') as f:
    f.write(str(-1))